<a href="https://colab.research.google.com/github/nvhieu0163/ComparativeOpinionMining_KLTN2023/blob/main/2_joint_CEE_SC_using_phoBERT_CRF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Tải thư viện

In [2]:
!pip install pytorch-crf

In [3]:
!pip install py_vncorenlp
!pip install transformers
!pip install tokenizers

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 8.2 MB/s eta 0:00:00
  Created wheel for py_vncorenlp: filename=py_vncorenlp-0.1.4-py3-none-any.whl size=4307 sha256=c9bb8b5ecf7ad4e9dbf72ba8c13d92a45c8c892c50077df83e22fd6fee6f29bf
  Stored in directory: /root/.cache/pip/wheels/d5/d9/bf/62632cdb007c702a0664091e92a0bb1f18a2fcecbe962d9827
Successfully built py_vncorenlp


## Load dữ liệu

In [4]:
cd "/content/drive/MyDrive/"

/content/drive/MyDrive


In [5]:
import json

In [6]:
list_of_stc = []
list_of_list_token_of_stc = []
list_of_list_label_of_stc = []
list_of_list_postag_of_stc = []

list_of_sentiment = []
list_of_stc_id = []

with open("./file12345_isPOStag_NER_phase_filter_UNK_oversampling.txt", "r", encoding = "utf-8") as f:
    while f:
        info = f.readline()[:-1] # bỏ dấu xuống dòng
        stc = f.readline()[:-1]
        token = f.readline()[:-1]
        label = f.readline()[:-1]
        postag = f.readline()[:-1]

        # check hết file
        if info == "" and stc == "" and label == "":
            break

        # split info
        info_split = info.split("\t")

        # add idx, add sentiment
        list_of_stc_id.append(int(info_split[0]))
        sentiment = json.loads(info_split[1])
        list_of_sentiment.append(sentiment)

        # if sentiment[0] == 0:
        #     list_of_sentiment.append([1])
        # elif sentiment[0] in [1, 2]:
        #     list_of_sentiment.append([2])
        # else:
        #     assert sentiment[0] in [-1, -2]
        #     list_of_sentiment.append([0])

        # add stc, add label
        list_of_stc.append(json.loads(stc))
        list_of_list_token_of_stc.append(json.loads(token))
        list_of_list_label_of_stc.append(json.loads(label))
        list_of_list_postag_of_stc.append(json.loads(postag))

In [7]:
len(list_of_sentiment)

9211

## Check các nhãn trong tập dữ liệu

In [8]:
# Check how many labels are there in the dataset
unique_labels = set()

for list_lb in list_of_list_label_of_stc:
  [unique_labels.add(lb) for lb in list_lb if lb not in unique_labels]

print(unique_labels)

{'B-sub', 'B-ap', 'I-obj', 'O', 'I-ap', 'B-obj', 'I-sub'}


## Load BERT Tokenizer

In [9]:
from transformers import AutoModel, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

## Mapping CEE label trước và sau khi BERT tokenize

In [10]:
MAX_LENGTH_SENTENCE = 256

In [11]:
# Padding
list_of_bert_padding_tokenized_stc = []
for stc in list_of_stc:
    list_of_bert_padding_tokenized_stc.append(tokenizer(stc, truncation=True ,padding='max_length' , max_length = MAX_LENGTH_SENTENCE, return_tensors="pt"))

In [12]:
# print(len(list_of_bert_padding_tokenized_stc[0]['input_ids'][0]))
# print(list_of_bert_padding_tokenized_stc[0])

In [13]:
# decode: Convert ids to token
list_of_bert_decode_padding_tokenized_stc = []

for tokenized_stc in list_of_bert_padding_tokenized_stc:
    list_of_bert_decode_padding_tokenized_stc.append(tokenizer.convert_ids_to_tokens(tokenized_stc["input_ids"][0]))

In [14]:
print(list_of_list_token_of_stc[0])
print(list_of_bert_decode_padding_tokenized_stc[0])

['Nhìn', 'vào', 'những', 'thông_số', 'kích_thước', 'trên', ',', 'VinFast_Lux_SA', '2.0', 'đang', 'nhỉnh', 'hơn', 'Kia_Sorento', 'về', 'mọi', 'chỉ_số', ',', 'điều', 'này', 'giúp', 'xe', 'vạm_vỡ', 'hơn', 'khi', 'nhìn', 'từ', 'phần', 'thân']
['<s>', 'Nhìn', 'vào', 'những', 'thông_số', 'kích_thước', 'trên', ',', 'Vin@@', 'Fast_@@', 'Lu@@', 'x_S@@', 'A', '2.0', 'đang', 'nhỉnh', 'hơn', 'Kia_@@', 'Sorento', 'về', 'mọi', 'chỉ_số', ',', 'điều', 'này', 'giúp', 'xe', 'vạm_vỡ', 'hơn', 'khi', 'nhìn', 'từ', 'phần', 'thân', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', 

In [15]:
def token_mapping_bert(bert_token_col, gold_token_col):
    """
    :param bert_token_col: a list of token list by BertTokenizer (with [cls] and [sep])
    :param gold_token_col: a list char list
    :return: a map: {bert_index: [char_index]}
    """
    assert len(bert_token_col) == len(gold_token_col), "bert data length not equal to char data length"

    mapping_col = []
    for index in range(len(bert_token_col)):
        seq_map, bert_index, token_index = {0: -1}, 1, 0    # Do bert_index nên bắt đầu từ 1, vì token đầu tiên là '<s>'
        seq_bert_token, seq_gold_token = bert_token_col[index], gold_token_col[index]   # khởi tạo seq_bert_token và seq_gold_token với giá trị index 0.

        while bert_index < len(seq_bert_token) and token_index < len(seq_gold_token):   # check chưa hết câu văn (không tính 'PAD')
            # print(index)
            seq_map[bert_index] = token_index

            # [UNK] denote special symbol
            if seq_bert_token[bert_index] == "<unk>":
                print(index)
                pass

            # get cur index correspond length
            token_length = len(seq_gold_token[token_index])     # độ dài của token
            bert_length = len(seq_bert_token[bert_index])       # độ dài của bert_token

            # drop "@@" prefix
            if seq_bert_token[bert_index].find("@@") != -1:     # tìm thấy subword
                bert_length = len(seq_bert_token[bert_index]) - 2   # sẽ không tính '@@', độ dài thực tế của bert_length phải trừ đi 2

            while token_length > bert_length:                       # chừng nào bert_length < token_length thì chưa đúng
                bert_index += 1
                seq_map[bert_index] = token_index
                bert_length += len(seq_bert_token[bert_index])
                if seq_bert_token[bert_index].find("@@") != -1:     # nếu vẫn còn subword, lặp lại quy trình
                    bert_length -= 2

            assert bert_length == token_length, "appear mapping error! {}?{}\n{}\n{}".format(bert_length,token_length,gold_token_col[index],bert_token_col[index])

            token_index = token_index + 1       # cập nhật index
            bert_index = bert_index + 1         # cập nhật index

        # Do độ dài seq_bert_token lớn hơn seq_gold_token 2 (thêm '<s>' và '</s>')
        seq_map[bert_index] = -1    # token_index đã out of range, còn bert_index sẽ là '</s>'

        mapping_col.append(seq_map)

    return mapping_col

In [16]:
mapping_dict = token_mapping_bert(list_of_bert_decode_padding_tokenized_stc, list_of_list_token_of_stc)

In [17]:
print(mapping_dict[282])

{0: -1, 1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 8: 7, 9: 8, 10: 9, 11: 10, 12: 11, 13: 12, 14: 13, 15: 14, 16: 15, 17: 16, 18: 16, 19: 17, 20: 18, 21: 19, 22: 20, 23: 21, 24: 22, 25: 23, 26: 24, 27: -1}


## Tạo dữ liệu

In [18]:
sub_labels = []
obj_labels = []
aspect_labels = []

for stc_idx in range(len(list_of_bert_padding_tokenized_stc)):    # _ là index cho câu thứ _ trong tập dữ liệu
    sub_label_stc = []
    obj_label_stc = []
    aspect_label_stc = []

    for bert_token_idx in range(MAX_LENGTH_SENTENCE):
        if bert_token_idx in mapping_dict[stc_idx].keys():      # CÓ TRONG CÂU. Tức không tính '<PAD>'
            gold_token_index = mapping_dict[stc_idx][bert_token_idx]

            if gold_token_index != -1:
                label_token = list_of_list_label_of_stc[stc_idx][gold_token_index]
                if label_token in ['I-sub', 'B-sub']:
                    sub_label_stc.append(1)
                    obj_label_stc.append(0)
                    aspect_label_stc.append(0)
                    continue
                elif label_token in ['I-obj','B-obj']:
                    sub_label_stc.append(0)
                    obj_label_stc.append(1)
                    aspect_label_stc.append(0)
                    continue
                elif label_token in ['I-ap', 'B-ap']:
                    sub_label_stc.append(0)
                    obj_label_stc.append(0)
                    aspect_label_stc.append(1)
                    continue
                else:
                    assert label_token == 'O', f'Không có label {label_token} trong câu'
                    sub_label_stc.append(0)
                    obj_label_stc.append(0)
                    aspect_label_stc.append(0)
            else:   # hai token đầu cuối
                assert list_of_bert_decode_padding_tokenized_stc[stc_idx][bert_token_idx] in ['<s>', '</s>']
                sub_label_stc.append(0)
                obj_label_stc.append(0)
                aspect_label_stc.append(0)

        else:       # CHỈ CÒN PAD
            assert list_of_bert_decode_padding_tokenized_stc[stc_idx][bert_token_idx] == '<pad>'
            sub_label_stc.append(0)
            obj_label_stc.append(0)
            aspect_label_stc.append(0)

    assert len(sub_label_stc) == len(obj_label_stc)     # đảm bảo label có độ dài như nhau
    assert len(obj_label_stc) == len(aspect_label_stc)
    assert len(sub_label_stc) == MAX_LENGTH_SENTENCE

    sub_labels.append(sub_label_stc)
    obj_labels.append(obj_label_stc)
    aspect_labels.append(aspect_label_stc)

In [19]:
dataset_dict = {
    'sub_labels': sub_labels,                # [[256], [256], [256], ..., [256], [256], [256]]
    'obj_labels': obj_labels,                # [[256], [256], [256], ..., [256], [256], [256]]
    'aspect_labels': aspect_labels,          # [[256], [256], [256], ..., [256], [256], [256]]
    'sentiment_labels': list_of_sentiment    # [[0]  , [1]  , [2],   ..., [0]  , [1]  , [1]  ]
}

In [20]:
dataset_list = []

for i in range(len(sub_labels)):
    dataset_list.append([sub_labels[i], obj_labels[i], aspect_labels[i], list_of_sentiment[i]])

# dataset_list  = [ [[256],[256],[256],[1]], [[256],[256],[256],[1]], [[256],[256],[256],[1]], [[256],[256],[256],[1]], ...... ]

In [21]:
print(dataset_list[0][0])
print(dataset_list[0][1])
print(dataset_list[0][2])
print(dataset_list[0][3])

[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

### Train/test dataset split

In [22]:
DATASET_SPLIT = 9000

In [23]:
X_train = list_of_bert_padding_tokenized_stc[:DATASET_SPLIT]   # [ {'input_idx': , 'attn_mask': }, {'input_idx': , 'attn_mask': },  {'input_idx': , 'attn_mask': }, ....]
y_train = dataset_list[:DATASET_SPLIT]                         # [ [[256],[256],[256],[1]]       , [[256],[256],[256],[1]]       ,  [[256], [256],[256],[1]],     , ....]
                                                                #     sub , obj , asp , s

X_test = list_of_bert_padding_tokenized_stc[DATASET_SPLIT:]
y_test = dataset_list[DATASET_SPLIT:]

In [24]:
len(y_test)

211

## Định nghĩa mô hình

In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchcrf import CRF

### Định nghĩa Dataloader

In [26]:
class DataSequence(torch.utils.data.Dataset):
    def __init__(self, tokenize_list, multi_label):
        self.tokenize = tokenize_list
        self.multi_label = multi_label

    def __len__(self):
        return len(self.tokenize)

    def __getitem__(self, idx):
        token_stc = self.tokenize[idx]
        sub, obj, aspect, sentiment = self.multi_label[idx]
        return token_stc, torch.LongTensor(sub), torch.LongTensor(obj), torch.LongTensor(aspect), torch.LongTensor(sentiment)

### Định nghĩa CRF layer

In [27]:
class CRFCell(nn.Module):
    def __init__(self, num_classes, batch_first=True):
        super(CRFCell, self).__init__()
        self.crf = CRF(num_tags=num_classes, batch_first=batch_first)

    def forward(self, feature, mask, target):
        mask = mask.bool()
        # feature : batch_size * seq_length * num_tags
        # mask : batch_size * seq_length
        # target : batch_size * seq_length

        # target is None means decode sequence
        if target is None:
            max_seq_len = feature.size(1)
            output = self.crf.decode(feature, mask)
            output = torch.as_tensor(
                [e + [0] * (max_seq_len - len(e)) for e in output],
                dtype=torch.long,
                device=feature.device
            )

        # target is not None means get loss
        else:
            output = self.crf(feature, target, mask, reduction='mean')
            output = output.neg()      # output: BATCH_SIZE
        return output

### Định nghĩa mô hình joint learning CEE + SC

In [28]:
# from transformers import BertForTokenClassification

class Baseline(torch.nn.Module):
    def __init__(self, model_type, model_config):
        super(Baseline, self).__init__()

        self.model_type = model_type
        self.gamma_sc = model_config.get('gamma_sc')
        self.gamma_cee = model_config.get('gamma_cee')
        self.encoder = AutoModel.from_pretrained('vinai/phobert-base')
        self.embedding_dropout = nn.Dropout(p = model_config.get('dropout_threshold'))

        self.sentiment_linear_1 = nn.Linear(self.encoder.config.hidden_size, 96)
        self.sentiment_linear_2 = nn.Linear(96, 3)         # better, equal, worse

        self.subject_linear =  nn.Linear(self.encoder.config.hidden_size, 2)    # O: outside, 1: Begin, 2: Inside
        self.object_linear =  nn.Linear(self.encoder.config.hidden_size, 2)
        self.aspect_linear = nn.Linear(self.encoder.config.hidden_size, 2)

        self.crf_subject_layer = CRFCell(2, batch_first = True)
        self.crf_object_layer = CRFCell(2, batch_first = True)
        self.crf_aspect_layer = CRFCell(2, batch_first = True)


    def forward(self, input_ids, attn_mask, sentiment_label=None, sub=None, obj=None, aspect=None):
        # get hidden representation (token embedding)
        token_embedding, pooled_output = self.encoder(input_ids, attn_mask, return_dict=False) # token_embedding = last_hidden_state
        batch_size, sequence_length, hidden_size = token_embedding.size()

        # drop out
        final_embedding = self.embedding_dropout(token_embedding)       # batch_size * sequence_length * hidden_size
        # class_embedding = self.embedding_dropout(pooled_output)

        # linear mapping.
        subject_linear_output = self.subject_linear(final_embedding)    # batch_size * sequence_length * num_label
        object_linear_output = self.object_linear(final_embedding)      # tức là bây h mỗi token chỉ được biểu diễn bởi vector 3 giá trị
        aspect_linear_output = self.aspect_linear(final_embedding)

        # get SC
        sentiment_avg_output = torch.mean(final_embedding, dim = 1)     # batch_size * hidden_size
        sentiment_linear_1_output = self.sentiment_linear_1(sentiment_avg_output)    # batch_size * 96
        sentiment_class_prob = self.sentiment_linear_2(sentiment_linear_1_output)     # batch_size * 3
        # sent_class_prob = self.sent_linear(class_embedding)

        # TESTING: Chỉ lấy đầu ra
        if sub is None and sentiment_label is None and obj is None and aspect is None:
            sentiment_class_output = sentiment_class_prob                       # batch_size * num_label

            subject_crf_output = self.crf_subject_layer(subject_linear_output, attn_mask, None)     # batch_size * sequence_length
            object_crf_output = self.crf_object_layer(object_linear_output, attn_mask, None)
            aspect_crf_output = self.crf_aspect_layer(aspect_linear_output, attn_mask, None)

            return sentiment_class_output, subject_crf_output, object_crf_output, aspect_crf_output

        # TRAINING: Lấy loss
        sentiment_loss = F.cross_entropy(sentiment_class_prob, sentiment_label.view(-1).to(device))

        sub_loss = self.crf_subject_layer(subject_linear_output, attn_mask, sub.to(device))
        obj_loss = self.crf_object_layer(object_linear_output, attn_mask, obj.to(device))
        asp_loss = self.crf_aspect_layer(aspect_linear_output, attn_mask, aspect.to(device))
        element_loss = sub_loss + obj_loss + asp_loss

        # according different model type to get different loss type.
        if self.model_type == "classification":
            return sentiment_loss
        elif self.model_type == "extraction":
            return element_loss
        else:
            assert self.model_type == 'joint'
            return self.gamma_sc * sentiment_loss + self.gamma_cee * element_loss

## Định nghĩa hàm Trainning

In [29]:
from torch.utils.data import DataLoader
from torch import optim
from tqdm import tqdm

def train(model, X_train, y_train, trainning_config):
    train_dataset = DataSequence(X_train, y_train)
    # val_dataset = DataSequence(X_val, y_val)

    train_dataloader = DataLoader(train_dataset, num_workers = 2, batch_size = trainning_config.get('batch_size'), shuffle = True)
    # val_dataloader = DataLoader(val_dataset, num_workers=2, batch_size=BATCH_SIZE)

    if use_cuda:
        model = model.cuda()

    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
        optimizer_need_model = model.module
    else:
        optimizer_need_model = model

    # Optimizer parameter defines
    optimizer = optim.Adam(
        [
            {'params': optimizer_need_model.encoder.parameters(), 'lr': trainning_config.get('bert_lr')},
            {'params': optimizer_need_model.sentiment_linear_1.parameters(), 'lr': trainning_config.get('sentiment_lr')},
            {'params': optimizer_need_model.sentiment_linear_2.parameters(), 'lr': trainning_config.get('sentiment_lr')},
            {'params': optimizer_need_model.subject_linear.parameters(), 'lr': trainning_config.get('element_lr')},
            {'params': optimizer_need_model.object_linear.parameters(), 'lr': trainning_config.get('element_lr')},
            {'params': optimizer_need_model.aspect_linear.parameters(), 'lr': trainning_config.get('element_lr')},
            {'params': optimizer_need_model.crf_subject_layer.parameters(), 'lr': trainning_config.get('crf_lr')},
            {'params': optimizer_need_model.crf_object_layer.parameters(), 'lr': trainning_config.get('crf_lr')},
            {'params': optimizer_need_model.crf_aspect_layer.parameters(), 'lr': trainning_config.get('crf_lr')}
        ],
        betas=(0.9, 0.99)
    )

    model.train()
    for epoch in range(trainning_config.get('epochs')):
        epoch_loss_train = 0

        for tokenize, sub, obj, aspect, sentiment in tqdm(train_dataloader):
            # print(tokenize)
            # print(feature_label)

            input_id = tokenize['input_ids'].squeeze(1).long().to(device)
            attn_mask = tokenize['attention_mask'].squeeze(1).long().to(device)

            sentiment.to(device)
            sub.to(device)
            obj.to(device)
            aspect.to(device)

            loss = model(input_id, attn_mask, sentiment, sub, obj, aspect)
            loss = torch.sum(loss)
            epoch_loss_train += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print("Epoch is {} and Loss: {:.2f}".format(epoch + 1, epoch_loss_train))

## Định nghĩa hàm Evaluating

### Định nghĩa hàm đánh giá sử dụng classification report

In [30]:
from sklearn.metrics import classification_report

In [31]:
from itertools import chain
def convert_list_of_list2list(nested_list):
    return list(chain(*nested_list))

In [32]:
def evaluate_metrics(y_label, y_pred):
    y_label_1D = convert_list_of_list2list(y_label)
    y_pred_1D = convert_list_of_list2list(y_pred)
    print('\n', classification_report(y_label_1D, y_pred_1D, digits=4))

### CEE: Định nghĩa hàm mapping output về gold_sequence

In [33]:
from collections import defaultdict

def mapping_output_to_actual_seq(y_label, y_pred, dataset_split = DATASET_SPLIT):
    assert len(y_label) == len(y_pred), 'Kích thước tập pred và tập test không giống nhau'
    actual_y_pred, actual_y_label = [], []

    for stc_index in range(len(y_label)):
        output = y_pred[stc_index]          # output hiện tại, kích thước 256
        label = y_label[stc_index]           # label hiện tại, kích thước 256
        assert len(output) == len(label)
        assert len(output) == MAX_LENGTH_SENTENCE, {f'Error: {len(output)}'}

        # chuyển đổi dict từ bert2gold về gold2bert
        dict_bert2gold = mapping_dict[dataset_split+stc_index]
        dict_gold2bert = defaultdict(list)
        for key, value in dict_bert2gold.items():
            if value == -1:
                continue
            dict_gold2bert[value].append(key)

        # bỏ padding
        trancated_output = output[:len(dict_bert2gold)]
        trancated_label = label[:len(dict_bert2gold)]

        pred, label = [], []
        for key in dict_gold2bert.keys():
            pred.append(trancated_output[dict_gold2bert[key][0]])
            label.append(trancated_label[dict_gold2bert[key][0]])

        assert len(pred) == len(list_of_list_token_of_stc[dataset_split+stc_index]), 'Đầu ra không bằng độ dài câu ban đầu'
        actual_y_pred.append(pred)
        actual_y_label.append(label)

    assert len(actual_y_pred) == len(y_pred)
    return actual_y_label, actual_y_pred

### CEE: Định nghĩa hàm đánh giá full-name entity

In [34]:
# Trả về 0 hoặc 1 nếu là exact hoặc binary
# Trả về tỉ lệ độ dài nếu là proportional

def get_first_index(stc : list, element_type):
    for index, e in enumerate(stc):
        if e == element_type:
            return index
    return -1


def get_last_index(stc : list, element_type):
    for index, e in enumerate(stc):
        if e == element_type and index < len(stc) -1:
            if stc[index+1] in ['O', 0]:
                return index
        elif e == element_type and index == len(stc) -1:
            return index
    return -1


def get_span_index(stc : list, element_type):
    return [get_first_index(stc, element_type), get_last_index(stc, element_type)]


def get_overlapping(span_test, span_pred):
    # span_test = [-1, -1]            # test
    # span_pred = [-1, -1]          # pred
    set1 = set(range(span_test[0], span_test[1] + 1))
    set2 = set(range(span_pred[0], span_pred[1] + 1))
    overlap = set1.intersection(set2)

    len_span_test = len(set1) if -1 not in set1 else 0  # không có span thì len_span_test là 0 chứ không phải là 1
    len_span_pred = len(set2) if -1 not in set2 else 0
    len_overlap = len(overlap) if ((-1 not in set1) and (-1 not in set2)) else 0 # Giải thích: cả 2 thằng đều là [-1, -1](không tồn tại) thì bản chất chúng không hề overlap

    # print(f'Kích thước test: {len_span_test}')
    # print(f'Kích thước pred: {len_span_pred}')
    # print(f'Kích thước overlap: {len_overlap}')

    return [len_span_test, len_span_pred, len_overlap]


def check_ovelapping_each_element(stc_test, stc_pred, element_type, evaluate_type):
    assert evaluate_type in ["exact", "partial", "binary"], "Wrong type of evaluate"
    assert element_type in [1, "SUB", "AP", "OBJ"], "Wrong type of element"
    assert len(stc_test) == len(stc_pred)

    span_test = get_span_index(stc_test, element_type)
    span_pred = get_span_index(stc_pred, element_type)
    len_span_test, len_span_pred, len_overlap = get_overlapping(span_test, span_pred)

    if evaluate_type == 'binary':
        if len_span_test == 0 and len_span_pred == 0:
            return 1
        elif len_overlap > 0 and len_span_test > 0 and len_span_pred > 0:       # chỉ cần có overlap là đúng rồi (bao gồm dù 1 có 1 ko có hay cả 2 cùng có)
            return 1
        return 0
    elif evaluate_type == 'exact':
        if len_span_test == 0 and len_span_pred == 0:
            return 1
        elif len_span_test == len_overlap and len_span_pred == len_overlap:
            return 1
        return 0
    elif evaluate_type == 'partial':
        if len_span_test == 0 and len_span_pred == 0:
            return [0, 0]                               # không ảnh hưởng gì, tỉ lệ đúng khi cộng vào vẫn được giữ nguyên
        elif len_overlap > 0 and len_span_test > 0 and len_span_pred > 0:
            return [len_overlap, len_span_test]
        return [-1, -1]
    else:
        raise Exception("No evaluate type found.")


def evaluate_full_named_entity(y_test, y_pred, element_type, evaluate_type, ovelapping_ratio = 0.75):
    assert evaluate_type in ["exact", "partial", "binary"], "Wrong type of evaluate"
    assert element_type in [1, "SUB", "AP", "OBJ"], "Wrong type of element"
    assert len(y_test) == len(y_pred)

    total_1_test, total_1_pred_true, total_1_pred = 0, 0, 0  #toàn bộ tập test

    for i in range(len(y_test)):
        assert len(y_test[i]) ==  len(y_pred[i]), 'Kích thước câu văn test và pred không bằng nhau'
        span_test = get_span_index(y_test[i], element_type)
        span_pred = get_span_index(y_pred[i], element_type)
        len_span_test, len_span_pred, len_overlap = get_overlapping(span_test, span_pred)

        if len_span_test > 0:   # Tức là nhãn có element này trong câu văn này
            total_1_test += 1

        if len_span_pred > 0:   # Tức là đoán có element
            total_1_pred +=1

        if len_span_pred > 0:   # y[0] !=-1 and y[1] != -1
            if evaluate_type == 'binary':
                if len_overlap > 0:
                    total_1_pred_true += 1

            elif evaluate_type == 'exact':
                if len_span_test == len_overlap and len_span_pred == len_overlap:
                    total_1_pred_true += 1

            elif evaluate_type == 'partial' and ovelapping_ratio is not None:
                if len_span_test != 0 and (len_overlap >= ovelapping_ratio * len_span_test):
                    total_1_pred_true += 1
            else:
                raise Exception("No evaluate type found. Or ovelapping_ratio is None")

    # print(total_1_test, total_1_pred, total_1_pred_true)
    precision = total_1_pred_true / total_1_pred
    recall = total_1_pred_true / total_1_test
    f1_score = 2*precision*recall/(precision+recall)
    # print(f'Precision: {round(precision, 4)*100}, Recall: {round(recall,4)*100}, F1-Score: {round(f1_score,4)*100}')
    print(f'Precision: {round(precision, 4)}, Recall: {round(recall,4)}, F1-Score: {round(f1_score,4)}')

### Định nghĩa hàm đánh giá all tuple

In [35]:
def check_correct_prediction_tuple(tuple_test, tuple_pred, evaluate_type):
    assert evaluate_type in ["exact", "partial", "binary"], "Wrong type of evaluate"

    sub_test, obj_test, aspect_test, sentiment_test = tuple_test
    sub_pred, obj_pred, aspect_pred, sentiment_pred = tuple_pred

    if sentiment_test != sentiment_pred:
        return 0

    if evaluate_type in ["exact", "binary"]:
        a = check_ovelapping_each_element(sub_test, sub_pred, 1, evaluate_type) # 0 hoặc 1 refer to False or True
        b = check_ovelapping_each_element(obj_test, obj_pred, 1, evaluate_type) # 0 hoặc 1
        c = check_ovelapping_each_element(aspect_test, aspect_pred, 1, evaluate_type) # 0 hoặc 1
        # print(a, b, c)
        if a + b + c == 3:
            return 1
        else:
            return 0
    elif evaluate_type == "partial":
        s1, s2  = check_ovelapping_each_element(sub_test, sub_pred, 1, evaluate_type)
        o1, o2 = check_ovelapping_each_element(obj_test, obj_pred, 1, evaluate_type)
        a1, a2 = check_ovelapping_each_element(aspect_test, aspect_pred, 1, evaluate_type)
        assert s1 <= s2
        assert o1 <= o2
        assert a1 <= a2

        if s1 == -1 or o1 == -1 or a1 == -1:
            return 0
        return (s1 + o1 + a1) / (s2 + o2 + a2)
    else:
        raise Exception("No evaluate type found.")
    return 0


def evaluate_all_tuple(all_label_dict, evaluate_type):
    num_correct_tuple = 0
    num_total_tuple = len(all_label_dict['SUBJECT']['test'])
    assert num_total_tuple == 211
    for i in range(num_total_tuple):
        tuple_test = [all_label_dict['SUBJECT']['test'][i], all_label_dict['OBJECT']['test'][i], all_label_dict['ASPECT']['test'][i], all_label_dict['SENTIMENT']['test'][i]]
        tuple_pred = [all_label_dict['SUBJECT']['pred'][i], all_label_dict['OBJECT']['pred'][i], all_label_dict['ASPECT']['pred'][i], all_label_dict['SENTIMENT']['pred'][i]]

        num_correct_tuple += check_correct_prediction_tuple(tuple_test, tuple_pred, evaluate_type)
        # print(num_correct_tuple)

    precision = num_correct_tuple / num_total_tuple
    recall = num_correct_tuple / num_total_tuple
    f1_score = 2*precision*recall/(precision+recall)
    print(f'Precision: {round(precision, 4)}, Recall: {round(recall,4)}, F1-Score: {round(f1_score,4)}')

### Định nghĩa hàm evaluate joint-learning

In [36]:
def evaluate_joint(model, X_test, y_test, testing_config):
    test_dataset = DataSequence(X_test, y_test)
    test_dataloader = DataLoader(test_dataset, num_workers = 2, batch_size = testing_config.get('batch_size'))

    if use_cuda:
        model = model.cuda()
    model.eval()
    with torch.no_grad():
        # khởi tạo torch rỗng
        dict_ = {
            "SUBJECT" : {'test': [], 'pred' : []},
            "OBJECT" : {'test': [], 'pred' : []},
            "ASPECT" : {'test': [], 'pred' : []},
            "SENTIMENT" : {'test': [], 'pred' : []}
        }

        for tokenize, sub, obj, aspect, sentiment in tqdm(test_dataloader):
            input_id = tokenize['input_ids'].squeeze(1).long().to(device)
            attn_mask = tokenize['attention_mask'].squeeze(1).long().to(device)

            sentiment.to(device)
            sub.to(device)
            obj.to(device)
            aspect.to(device)

            sent_output_pred, sub_pred, obj_pred, aspect_pred = model(input_id, attn_mask)
            _, sent_output_pred = torch.max(torch.softmax(sent_output_pred, dim=1), dim=1) # Batchsize

            dict_['SENTIMENT']['pred'].extend(sent_output_pred.type(torch.int64).cpu().numpy())
            dict_['SENTIMENT']['test'].extend(sentiment.type(torch.int64).cpu().numpy())

            for batch in range(len(sub_pred)):
                dict_['SUBJECT']['pred'].append(sub_pred[batch].type(torch.int64).cpu().numpy())
                dict_['OBJECT']['pred'].append(obj_pred[batch].type(torch.int64).cpu().numpy())
                dict_['ASPECT']['pred'].append(aspect_pred[batch].type(torch.int64).cpu().numpy())

                dict_['SUBJECT']['test'].append(sub[batch].type(torch.int64).cpu().numpy())
                dict_['OBJECT']['test'].append(obj[batch].type(torch.int64).cpu().numpy())
                dict_['ASPECT']['test'].append(aspect[batch].type(torch.int64).cpu().numpy())

    print("\n------------------------------SENTIMENT EVALUATION--------------------------------------")
    print(classification_report(dict_['SENTIMENT']['test'], dict_['SENTIMENT']['pred'], digits=4))

    for element_type in ["SUBJECT", "OBJECT", "ASPECT"]:
        print(f"\n------------------------------{element_type} EVALUATION--------------------------------------")

        y_label, y_pred = mapping_output_to_actual_seq(dict_[element_type]['test'], dict_[element_type]['pred'])
        for evaluate_type in ["binary", "partial", "exact"]:
            print(f'{evaluate_type.upper()}', end = ':  ')
            evaluate_full_named_entity(y_label, y_pred, 1, evaluate_type)

        evaluate_metrics(y_label, y_pred)

    print("\n------------------------------FINAL TUPLE EVALUATION--------------------------------------")
    for evaluate_type in ["binary", "partial", "exact"]:
        print(f'{evaluate_type.upper()}', end = ':  ')
        evaluate_all_tuple(dict_, evaluate_type)


In [42]:
def evaluate_sequential(model_sc, model_cee, X_test, y_test, testing_config):
    test_dataset = DataSequence(X_test, y_test)
    test_dataloader = DataLoader(test_dataset, num_workers = 2, batch_size = testing_config.get('batch_size'))

    if use_cuda:
        model_sc = model_sc.cuda()
        model_cee = model_cee.cuda()
    model_sc.eval()
    model_cee.eval()
    with torch.no_grad():
        # khởi tạo torch rỗng
        dict_ = {
            "SUBJECT" : {'test': [], 'pred' : []},
            "OBJECT" : {'test': [], 'pred' : []},
            "ASPECT" : {'test': [], 'pred' : []},
            "SENTIMENT" : {'test': [], 'pred' : []}
        }

        for tokenize, sub, obj, aspect, sentiment in tqdm(test_dataloader):
            input_id = tokenize['input_ids'].squeeze(1).long().to(device)
            attn_mask = tokenize['attention_mask'].squeeze(1).long().to(device)

            sentiment.to(device)
            sub.to(device)
            obj.to(device)
            aspect.to(device)

            sent_output_pred, _, _, _ = model_sc(input_id, attn_mask)
            _, sent_output_pred = torch.max(torch.softmax(sent_output_pred, dim=1), dim=1) # Batchsize
            _, sub_pred, obj_pred, aspect_pred = model_cee(input_id, attn_mask)

            dict_['SENTIMENT']['pred'].extend(sent_output_pred.type(torch.int64).cpu().numpy())
            dict_['SENTIMENT']['test'].extend(sentiment.type(torch.int64).cpu().numpy())

            for batch in range(len(sub_pred)):
                dict_['SUBJECT']['pred'].append(sub_pred[batch].type(torch.int64).cpu().numpy())
                dict_['OBJECT']['pred'].append(obj_pred[batch].type(torch.int64).cpu().numpy())
                dict_['ASPECT']['pred'].append(aspect_pred[batch].type(torch.int64).cpu().numpy())

                dict_['SUBJECT']['test'].append(sub[batch].type(torch.int64).cpu().numpy())
                dict_['OBJECT']['test'].append(obj[batch].type(torch.int64).cpu().numpy())
                dict_['ASPECT']['test'].append(aspect[batch].type(torch.int64).cpu().numpy())

    print("\n------------------------------SENTIMENT EVALUATION--------------------------------------")
    print(classification_report(dict_['SENTIMENT']['test'], dict_['SENTIMENT']['pred'], digits=4))

    for element_type in ["SUBJECT", "OBJECT", "ASPECT"]:
        print(f"\n------------------------------{element_type} EVALUATION--------------------------------------")

        y_label, y_pred = mapping_output_to_actual_seq(dict_[element_type]['test'], dict_[element_type]['pred'])
        for evaluate_type in ["binary", "partial", "exact"]:
            print(f'{evaluate_type.upper()}', end = ':  ')
            evaluate_full_named_entity(y_label, y_pred, 1, evaluate_type)

        evaluate_metrics(y_label, y_pred)

    print("\n------------------------------FINAL TUPLE EVALUATION--------------------------------------")
    for evaluate_type in ["binary", "partial", "exact"]:
        print(f'{evaluate_type.upper()}', end = ':  ')
        evaluate_all_tuple(dict_, evaluate_type)

## Main

### Running main

In [38]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
!mkdir -p phase2_saved_model

In [39]:
MODE_PROGRAM = "JOINT"

if MODE_PROGRAM == 'JOINT':
    # Khởi tạo
    model_config = {
        'gamma_sc' : 1.1,
        'gamma_cee' : 1,
        'dropout_threshold' : 0.2
    }
    model = Baseline(model_type = 'joint', model_config = model_config)

    # trainning
    trainning_config = {
        'epochs' : 5,
        'batch_size' : 32,
        'bert_lr' : 2e-5,
        'sentiment_lr' : 2e-3,
        'element_lr' : 2e-5,
        'crf_lr' : 5e-3
    }
    train(model, X_train, y_train, trainning_config)

    # testing
    testing_config = {
        'batch_size' : 32
    }
    evaluate_joint(model, X_test, y_test, testing_config)
    torch.save(model.state_dict(), 'phase2_saved_model/joint_model.pth')

pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

100%|██████████| 282/282 [09:42<00:00,  2.07s/it]


Epoch is 1 and Loss: 2171.71


100%|██████████| 282/282 [09:44<00:00,  2.07s/it]


Epoch is 2 and Loss: 488.43


...

Epoch is 4 and Loss: 225.76


100%|██████████| 282/282 [09:42<00:00,  2.07s/it]


Epoch is 5 and Loss: 118.21


100%|██████████| 7/7 [00:03<00:00,  1.91it/s]



------------------------------SENTIMENT EVALUATION--------------------------------------
              precision    recall  f1-score   support

           0     0.8537    0.9211    0.8861        38
           1     0.8810    0.9250    0.9024        40
           2     0.9609    0.9248    0.9425       133

    accuracy                         0.9242       211
   macro avg     0.8985    0.9236    0.9103       211
weighted avg     0.9265    0.9242    0.9248       211


------------------------------SUBJECT EVALUATION--------------------------------------
BINARY:  Precision: 0.9804, Recall: 0.9756, F1-Score: 0.978
PARTIAL:  Precision: 0.9706, Recall: 0.9659, F1-Score: 0.9682
EXACT:  Precision: 0.9216, Recall: 0.9171, F1-Score: 0.9193

               precision    recall  f1-score   support

           0     0.9976    0.9922    0.9949      4516
           1     0.9266    0.9757    0.9505       453

    accuracy                         0.9907      4969
   macro avg     0.9621    0.9840    0.

In [40]:
del model

In [ ]:
MODE_PROGRAM = "SEQUENTIAL"
if MODE_PROGRAM == 'SEQUENTIAL':
    # Khởi tạo
    model_config = {
        'gamma_sc' : 0,
        'gamma_cee' : 0,
        'dropout_threshold' : 0.2
    }
    model_sc = Baseline(model_type = 'classification', model_config = model_config)
    model_cee = Baseline(model_type = 'extraction', model_config = model_config)

    # trainning
    trainning_config = {
        'epochs' : 5,
        'batch_size' : 32,
        'bert_lr' : 2e-5,
        'sentiment_lr' : 2e-4,
        'element_lr' : 2e-5,
        'crf_lr' : 5e-3
    }
    train(model_sc, X_train, y_train, trainning_config)
    train(model_cee, X_train, y_train, trainning_config)

    # testing
    testing_config = {
        'batch_size' : 32
    }
    torch.save(model_sc.state_dict(), 'phase2_saved_model/sc_model.pth')
    torch.save(model_cee.state_dict(), 'phase2_saved_model/cee_model.pth')

In [45]:
evaluate_sequential(model_sc, model_cee, X_test, y_test, testing_config)

100%|██████████| 7/7 [00:08<00:00,  1.25s/it]



------------------------------SENTIMENT EVALUATION--------------------------------------
              precision    recall  f1-score   support

           0     0.8718    0.8947    0.8831        38
           1     1.0000    0.9250    0.9610        40
           2     0.9481    0.9624    0.9552       133

    accuracy                         0.9431       211
   macro avg     0.9400    0.9274    0.9331       211
weighted avg     0.9442    0.9431    0.9433       211


------------------------------SUBJECT EVALUATION--------------------------------------
BINARY:  Precision: 0.9755, Recall: 0.9707, F1-Score: 0.9731
PARTIAL:  Precision: 0.951, Recall: 0.9463, F1-Score: 0.9487
EXACT:  Precision: 0.9314, Recall: 0.9268, F1-Score: 0.9291

               precision    recall  f1-score   support

           0     0.9945    0.9942    0.9944      4516
           1     0.9427    0.9448    0.9438       453

    accuracy                         0.9897      4969
   macro avg     0.9686    0.9695    0.

In [ ]:
del model_sc
del model_cee

### Lưu model

In [ ]:
#
# tokenizer.save_pretrained(f'phase2_saved_model/tokenizer')
# if MODE_PROGRAM == 'JOINT':
#     model.save_pretrained('phase2_saved_model/joint_model')
# else:
#

In [ ]:
# if MODEL_TYPE == "classification":
#     !mkdir -p saved_model
#     torch.save(cls.state_dict(), 'saved_model/SC')

# if MODEL_TYPE == "extraction":
#     !mkdir -p saved_model
#     torch.save(cee.state_dict(), 'saved_model/CEE')

In [ ]:
# model.bert.save_pretrained('saved_model/bertforNER')

## Thực nghiệm

In [ ]:
# prediction = []
# BATCHSIZE = 20
# num_class = 3
# sequence_length = 3

# for i in range(10):
#     input = torch.randn(BATCHSIZE, num_class)
#     output_softmax = torch.softmax(input, dim=1)
#     # print('-------')
#     # print(output_softmax.shape)
#     # print(output_softmax)
#     _, output_max = torch.max(output_softmax, dim=1)
#     # print(output_max)
#     # break
#     prediction.extend(output_max.cpu().numpy())

# print(prediction)

# for i in range(10):
#     output = torch.randn(BATCHSIZE, sequence_length)
    # output_softmax = torch.softmax(output, dim=1)
    # print('-------')
    # print(output_softmax.shape)
    # print(output_softmax)
    # _, output_max = torch.max(output_softmax, dim=1)
    # print(output_max)
    # break
    # print(len(output))
    # for i, stc in enumerate(output):
    #     # print(i)
    #     prediction.append(stc.cpu().numpy())

# print(prediction)

In [ ]:
# import torch
# import torch.nn.functional as F

# x = torch.tensor([1.0, 2.0, 3.0, 4.0, 5.0, 6.0])
# x = x.view(1, 1, -1)  # Adding batch and channel dimensions for 1D convolution
# print(x)
# pooled_x = F.avg_pool1d(x, kernel_size=2, stride=1)
# print(pooled_x)

In [ ]:
# import torch

# Assuming you have token_embeddings with shape (batch_size, sequence_length, hidden_size)
# For example, you may get this from BERT using the Hugging Face Transformers library

# Mock token embeddings (replace this with actual token embeddings)
# token_embeddings = torch.rand((2, 10, 768))  # Assuming batch_size=2, sequence_length=10, hidden_size=768
# attn_mask = torch.rand((2, 10)) * 0 + 1

# linear = nn.Linear(768, 2)
# output = linear(token_embeddings)
# print(f'After linear shape: {output.shape}')

# crf_layer = CRFCell(2, batch_first = True)
# output = crf_layer(output, attn_mask, None)
# print(f'After crf shape: {output.shape}')
# print(output)
# Apply mean pooling along the sequence dimension
# mean_pooled_output = torch.mean(output, dim=1)

# token_embeddings = torch.mean(token_embeddings, dim = 1)
# output_after_avg = avg_layer(token_embeddings, dim = 1)
# print(token_embeddings.shape)


# print("Token Embeddings shape:", token_embeddings.shape)
# print("Mean Pooled Output shape:", mean_pooled_output.shape)
# print(mean_pooled_output)
# print(mean_pooled_output.view(-1).shape)
# print(mean_pooled_output.view(-1))
# print(mean_pooled_output.size(1))